# AUTOMATIZACION DE WEB SCRAPING
Se busca automatizar el proceso para scrapear página de la página 'El País'

In [1]:
# Importamos las bibliotecas necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def url_reader(url, type_url = 'text'):
    print(f'El Sitio es: {url}')
    try:
        site_connection = requests.get(url)
        print(f'El Status Code es: {site_connection.status_code}')
        if site_connection.status_code != 200: 
            return None
        else:
            if type_url == 'text':
                print(f'Se retorna tipo: {type_url}')
                return BeautifulSoup(site_connection.text, 'lxml')
            else:
                print(f'Se retorna tipo: {type_url}')
                return BeautifulSoup(site_connection.content, 'lxml')
    except Exception as e:
        print(f'Se econtro el siguiente error: {e}')
        return None

In [3]:
url_site = 'https://www.pagina12.com.ar'

## Obtenemos las Secciones del Sitio 

In [4]:
def get_sections(url):
    main_site = url_reader(url)
    try: 
        if main_site:
            print('Obteniendo Secciones')
            sections = main_site.find('div', attrs = 'p12-dropdown-column').find_all('a')
            sections_names = [section.get_text() for section in sections]
            sections_urls = [section.get('href') for section in sections]
            new_sections = {sections_names[i] : sections_urls[i] for i in range(len(sections_names))}
            return new_sections
        else:
            print(f'No se puede Scrapear: {url}')
            return None
    except Exception as e:
        print(f'Se encontro el siguiente error: {e}')
        return None

## Obtenemos las Notas de cada Seccion

In [5]:
def url_creator(url):
    return lambda complement : url + complement if complement[0] == '/' else complement

site_creator = url_creator(url_site)

In [6]:
def get_news(name, url):
    section_site = url_reader(url)
    try:
        if section_site:
            print(f'Obteniendo Noticias de la Sección: {name}')
            news_links = []
            primary_news = section_site.find_all('h2', attrs = {'class': 'title-list'})
            secundary_news = section_site.find_all('h3', attrs = {'class': 'title-list'})
            other_news = section_site.find_all('h4', attrs = {'class': 'title-list'})
            if len(primary_news) != 0:
                for new in primary_news:
                    news_links.append(site_creator(new.a.get('href')))
            if len(secundary_news) != 0:
                for new in secundary_news:
                    news_links.append(site_creator(new.a.get('href')))
            if len(other_news) != 0:
                for new in other_news:
                    news_links.append(site_creator(new.a.get('href')))
            return news_links
        else:
            print(f'No se puede Scrapear la seccion: {name}')
            return None
    except Exception as e:
        print(f'Se encontro el siguiente error: {e}')
        return None

In [7]:
def get_all_news(url):
    sections = get_sections(url)
    all_news = []
    for k, v in sections.items():
        for new in get_news(k, v):
            all_news.append(new)
    return all_news

## Obtenemos la Información de cada Nota

In [8]:
def get_information(url):
    new_site = url_reader(url)
    information = dict()
    try:
        if new_site:
            section = new_site.find('h5', attrs = {'class': 'current-tag'})
            if section:
                information['Section'] = section.a.get_text()
            else:
                information['Section'] = None
            title = new_site.find('h1')
            if title:
                information['Title'] = title.get_text()
            else:
                information['Title'] = None
            subtitle = new_site.find('div', attrs = {'class': 'col 2-col'}).h4
            if subtitle:
                information['Subtitle'] = subtitle.get_text()
            else:
                information['Subtitle'] = None
            author = new_site.find('div', attrs = {'class' : 'author-name'})
            if author:
                information['Author'] = author.get_text()
            else:
                information['Author'] = None
            date = new_site.find('time')
            if date:
                information['Date'] = date.get_text()
            else:
                information['Date'] = None
            information['URL'] = url
            return information
        else:
            information['Section'] = None
            information['Title'] = None
            information['Subtitle'] = None
            information['Author'] = None
            information['Date'] = None
            information['URL'] = url
            return information
    except Exception as e:
        print(f'Se encontro el siguiente error: {e}')
        information['Section'] = None
        information['Title'] = None
        information['Subtitle'] = None
        information['Author'] = None
        information['Date'] = None
        information['URL'] = url
        return information

In [9]:
def get_all_information(url):
    news = get_all_news(url)
    information = []
    for new in news:
        information.append(get_information(new))
    return information

In [10]:
information = get_all_information(url_site)

El Sitio es: https://www.pagina12.com.ar
El Status Code es: 200
Se retorna tipo: text
Obteniendo Secciones
El Sitio es: https://www.pagina12.com.ar/secciones/el-pais
El Status Code es: 200
Se retorna tipo: text
Obteniendo Noticias de la Sección: El país
El Sitio es: https://www.pagina12.com.ar/secciones/economia
El Status Code es: 200
Se retorna tipo: text
Obteniendo Noticias de la Sección: Economía
El Sitio es: https://www.pagina12.com.ar/secciones/sociedad
El Status Code es: 200
Se retorna tipo: text
Obteniendo Noticias de la Sección: Sociedad
El Sitio es: https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos
El Status Code es: 200
Se retorna tipo: text
Obteniendo Noticias de la Sección: Espectáculos
El Sitio es: https://www.pagina12.com.ar/secciones/deportes
El Status Code es: 200
Se retorna tipo: text
Obteniendo Noticias de la Sección: Deportes
El Sitio es: https://www.pagina12.com.ar/secciones/ciencia
El Status Code es: 200
Se retorna tipo: text
Obteniendo Noticias de la 

El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/440364-palermo-tres-detenidos-por-escalar-a-un-balcon-para-robar
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/440363-lucas-gonzalez-piden-el-sobreseimiento-de-la-policia-lujan-s
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/440220-efemerides-de-hoy-que-paso-un-27-de-julio
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/440484-boca-juniors-dos-buenas-noticias-para-el-negro-ibarra
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/440482-la-situacion-de-fernandez-mercau-en-san-lorenzo
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/440468-la-familia-de-passarella-desmintio-los-rumores-sobre-su-esta
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/440462-la-venta-de-giuliano-galo

El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/713-los-genios-locos-del-ajedrez
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/376577-vic-morrow-el-actor-que-marco-una-epoca-con-combate-y-murio-
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/439284-nirvana-vuelve-a-escalar-en-las-listas-de-reproducciones-gra
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/439189-hallaron-muerta-a-una-actriz-de-elvis
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/438834-la-historia-de-callejero-fino-el-rapero-que-se-reinvento-en-
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/438248-el-indio-solari-publica-cuatro-canciones-del-show-de-los-fun
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/437419-steven-spielberg-al-frente-de-una-pelicul

El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/284335-un-malbec-mendocino-fue-elegido-como-el-mejor-del-mundo-y-re
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/392363-musica-esencial-ludmila-fernandez-y-roman-ostrowski
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/392360-musica-esencial-jorge-correa
El Status Code es: 429
El Sitio es: https://www.pagina12.com.ar/392357-musica-esencial-jorge-lagos
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/392356-musica-esencial-mariano-bongiorno
El Status Code es: 429
El Sitio es: https://www.pagina12.com.ar/392355-musica-esencial-mariela-orda
El Status Code es: 429
El Sitio es: https://www.pagina12.com.ar/392354-musica-esencial-rolando-de-marco
El Status Code es: 429
El Sitio es: https://www.pagina12.com.ar/415947-afrontar-el-cambio-de-epoca-la-idea-de-futuro-una-mirada-des
El Status Code es

El Status Code es: 429
El Sitio es: https://www.pagina12.com.ar/439083-quintela-anuncio-un-aumento-de-6-000-pesos-para-estatales-y-
El Status Code es: 429
El Sitio es: https://www.pagina12.com.ar/438750-hasta-el-momento-8-800-riojanos-se-inscribieron-en-el-regist
El Status Code es: 429
El Sitio es: https://www.pagina12.com.ar/240289-el-brindis-con-los-socios-de-pagina-12
El Status Code es: 429
El Sitio es: https://www.pagina12.com.ar/240286-lo-mejor-de-argentina-piensa-parte-1
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/240283-un-nomade-llamado-gianluca
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/242804-nicolas-trotta-la-escuela-debe-recuperar-su-rol-central
El Status Code es: 429
El Sitio es: https://www.pagina12.com.ar/235988-d-onofrio-se-pasa-de-bando
El Status Code es: 200
Se retorna tipo: text
El Sitio es: https://www.pagina12.com.ar/235981-dragon-rojo-porteno
El Status Code es: 200
Se retorna tipo

In [11]:
df = pd.DataFrame.from_records(information)

In [13]:
df.to_csv('Noticias')

In [15]:
df.head(40)

,Section,Title,Subtitle,Author,Date,URL
0,El país,"Alberto Fernández, sobre el Dólar Soja: ""Es un...","""Nosotros no vamos a devaluar""",None,27 de julio de 2022 - 19:27,https://www.pagina12.com.ar/440486-alberto-fer...
1,El país,"El Monumento al Descamisado, la ambiciosa obra...",Un mausoleo del peronismo en plena Recoleta,None,27 de julio de 2022 - 19:07,https://www.pagina12.com.ar/440445-el-monument...
2,El país,Mauricio Macri no disimula su deseo de volver ...,"""Hay que retomar el rumbo de los años 90 y en ...",None,27 de julio de 2022 - 17:14,https://www.pagina12.com.ar/440429-mauricio-ma...
3,El país,Alberto Fernández convocó a los gobernadores e...,Almuerzo en la Casa Rosada,Por Melisa Molina,27 de julio de 2022 - 18:41,https://www.pagina12.com.ar/440369-alberto-fer...
4,El país,El lazo que une a Luis Etchevehere con el rura...,Dolores Etchevehere habló del entramado que lo...,None,27 de julio de 2022 - 13:46,https://www.pagina12.com.ar/440362-el-lazo-que...
5,El país,"""Todo vuelve"", el sugestivo video que publicó ...",En medio de rumores de cambios en el gabinete,None,27 de julio de 2022 - 12:45,https://www.pagina12.com.ar/440353-todo-vuelve...
6,El país,Subsidios de luz y gas: el Gobierno extendió l...,Video tutorial del trámite para mantener los s...,None,27 de julio de 2022 - 11:18,https://www.pagina12.com.ar/440050-subsidios-d...
7,El país,"Grabois criticó el anuncio del ""dólar soja"": ""...",Vía Twitter,None,27 de julio de 2022 - 15:36,https://www.pagina12.com.ar/440326-grabois-cri...
8,El país,"Daer: ""La luz de Evita nos marca el horizonte""",Acto en la CGT por el 70 aniversario del falle...,None,27 de julio de 2022 - 08:00,https://www.pagina12.com.ar/440260-daer-la-luz...
9,El país,Insaurralde y Vilar firmaron convenios marco p...,Gestión del Ministerio de Ambiente,None,27 de julio de 2022 - 16:34,https://www.pagina12.com.ar/440408-insaurralde...
